In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F


In [0]:
dbutils.widgets.text("catalogo", "catalog_dev")
dbutils.widgets.text("esquema_source", "silver")
dbutils.widgets.text("esquema_sink", "golden")

In [0]:
catalogo = dbutils.widgets.get("catalogo")
esquema_source = dbutils.widgets.get("esquema_source")
esquema_sink = dbutils.widgets.get("esquema_sink")

In [0]:
df_orders = spark.table(f"{catalogo}.{esquema_source}.orders_transformed")
df_customer_agg = (
    df_orders.groupBy("user_id").agg(countDistinct("order_id").alias("total_orders"),
        avg("days_since_prior_order").alias("avg_days_between_orders")
    )
)

In [0]:
dim_customer = (
    df_customer_agg.select(col("user_id"),col("total_orders"),col("avg_days_between_orders"))
)

In [0]:
dim_customer.write.mode("overwrite").saveAsTable(f"{catalogo}.{esquema_sink}.dim_customer")